In speech processing we may use really huge networks cause majority of the processing bottleneck is on CPU and running the CTC function...

Also, CLARIN mobile corpus seems to be too small for advanced network

In [16]:
#!/usr/bin/python3

# For demonstration purposes - Paweł Tomasik

# for CLARIN_MOBILE - generally it is unnormalized

class TrainingGenerator:
    """
    Loads data in batches. It has well defined shape of an array into which load. At the beginning of training,
    loads new ordering of samples that are in multiple files. Using memmap, load the selected data into the back buffer.
    Self.ordering
    
    Switches between two banks, using one bank while filling the other in the background thread.
    ...    
    """

    def __init__(self, X_files, Y_files, buffer_limit_size=4*(2**30), batch_size):
        """Assuming that X_files and Y_files are corresponding"""
        self.front_buffer = None
        self.back_buffer = None
        self.X_files = X_files
        self.Y_files = Y_files
        samples, times = [], []
        for file in X_files:
            data = np.load(file, mmap_mode='r')
            n_sample, time, fftwidth = data.shape
            dtype = data.dtype
            samples.append(n_sample)
            times.append(time)
        rec_time = max(times)
        self.samples_per_file = samples
        size_of_recording = 4 * times * fftwidth
        self.n_records_in_buffer = buffer_limit_size // size_of_recording
        self.batch_size = batch_size
        self.buffer_slider = 0
        # initialize the front buffer, run the back buffer
                
    def new_epoch(self):
        permutations = np.stack([
            np.stack([np.ones(samples) * recording, np.arange(samples)]).T
            for recording, samples in self.samples_per_file
        ], axis=0)
        self.permutation = np.random.permute(permutations)
    
    def __next__(self):
        if self.buffer_slider + self.batch_size < self.n_records_in_buffer:
            returnable = self.front_buffer[self.buffer_slider : self.buffer_slider + self.batch_size]
            self.buffer_slider += self.batch_size
            return returnable
        else:
            """
            Create the return buffer
            Fill first recordings from the old front buffer
            Switch buffers, trigger the buffer replacement
            Get the last recordings from the beginning of the new buffer
            Return the buffer
            """



5523 5536


In [9]:
np.stack([np.ones(10), np.arange(10)])

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.]])

In [9]:
(X_lens >= Y_lens).mean()

0.9828605914302957

In [9]:
(X_lens == 0).mean()

0.0